In [ ]:
!pip install biopython scikit-learn --quiet

import pandas as pd
import numpy as np
from Bio import SeqIO
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from random import shuffle

def load_sequences(file_paths, window=150, stride=50):
    sequences = []
    labels = []
    for label, path in file_paths.items():
        for record in SeqIO.parse(path, "fasta"):
            seq = str(record.seq).replace("*", "")
            for i in range(0, len(seq) - window + 1, stride):
                frag = seq[i:i+window]
                if len(frag) == window:
                    sequences.append(frag)
                    labels.append(label)
    combined = list(zip(sequences, labels))
    shuffle(combined)
    sequences, labels = zip(*combined)
    return list(sequences), list(labels)

def get_kmers(seq, k=3):
    return [seq[i:i+k] for i in range(len(seq)-k+1)]

def build_kmer_features(sequences, k=3):
    all_kmers = set()
    kmer_counts = []
    for seq in sequences:
        kmers = get_kmers(seq, k)
        count = Counter(kmers)
        kmer_counts.append(count)
        all_kmers.update(count)
    all_kmers = sorted(list(all_kmers))
    df = pd.DataFrame(0, index=np.arange(len(sequences)), columns=all_kmers)
    for i, count in enumerate(kmer_counts):
        for kmer, freq in count.items():
            df.at[i, kmer] = freq
    return df

file_paths = {
    'Zika': '/content/drive/MyDrive/All_Disease_Protein_Classes/Zika/Zika_proteins.fasta',
    'Ebola': '/content/drive/MyDrive/All_Disease_Protein_Classes/Ebola/Ebola_proteins.fasta',
    'SARS_CoV_2': '/content/drive/MyDrive/All_Disease_Protein_Classes/SARS_CoV_2/SARS_CoV_2_proteins.fasta',
    'Influenza_A': '/content/drive/MyDrive/All_Disease_Protein_Classes/Influenza_A/Influenza_A_proteins.fasta',
    'Influenza_B': '/content/drive/MyDrive/All_Disease_Protein_Classes/Influenza_B/Influenza_B_proteins.fasta',
    'Tuberculosis': '/content/drive/MyDrive/All_Disease_Protein_Classes/Tuberculosis/Tuberculosis_proteins.fasta',
    'Human': '/content/drive/MyDrive/All_Disease_Protein_Classes/Normal/Human_proteinss.fasta'
}

data, labels = load_sequences(file_paths, window=150, stride=50)
print(f" Augmented sequences: {len(data)}")

X = build_kmer_features(data, k=3)
y = pd.factorize(pd.Series(labels))[0]

selector = SelectKBest(mutual_info_classif, k=180)
X_selected = selector.fit_transform(X, y)

X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, stratify=y, random_state=42
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
svm_poly = SVC(
    kernel='poly',
    degree=3,
    C=2.4,
    gamma=0.001,
    coef0=1,
    class_weight='balanced',
    random_state=42
)
svm_poly.fit(X_train_scaled, y_train)

y_pred = svm_poly.predict(X_test_scaled)
acc = accuracy_score(y_test, y_pred)

print(f"\n Final SVM-Poly Accuracy: {acc * 100:.2f}%")
print("\n Classification Report:\n")
print(classification_report(y_test, y_pred))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 20.5 MB/s eta 0:00:00
 Augmented sequences: 116

 Final SVM-Poly Accuracy: 95.83%

 Classification Report:

              precision    recall  f1-score   support

           0       0.90      1.00      0.95         9
           1       1.00      0.86      0.92         7
           2       1.00      1.00      1.00         1
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         3
           6       1.00      1.00      1.00         1

    accuracy                           0.96        24
   macro avg       0.99      0.98      0.98        24
weighted avg       0.96      0.96      0.96        24

